In [1]:
flower_mapping = {'daisy':0, 'dandelion':1, 'rose':2, 'sunflower':3, 'tulip':4}
train_dir = "ml100-03-final/image_data/train/"
test_dir = "ml100-03-final/image_data/test/test"

In [2]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    validation_split=0.1,
    rescale=1./255,
#     rotation_range=40,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True
)

Using TensorFlow backend.


In [3]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training',
)
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
)

Found 2543 images belonging to 5 classes.
Found 280 images belonging to 5 classes.


In [4]:
# import matplotlib.pyplot as plt
# import numpy as np
# %matplotlib inline
# for data_batch, labels_batch in train_generator:
#     print('data batch shape: ', data_batch.shape)
#     print('labels batch shape: ', labels_batch.shape)
#     print(labels_batch[1])
#     plt.imshow(data_batch[1])
#     break

In [5]:
import numpy as np
img_shape = train_generator.target_size
num_classes = len(np.unique(train_generator.classes))
print(img_shape, num_classes)

(150, 150) 5


In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [7]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=img_shape+(3,), kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), padding='same', kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'))

for n in [64, 128, 256]:
    model.add(Conv2D(n, (3, 3), padding='same', kernel_initializer='he_normal'))
    model.add(Activation('relu'))
    model.add(Conv2D(n, (3, 3), padding='same', kernel_initializer='he_normal'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'))
    model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(units=128, kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, kernel_initializer='he_normal'))
model.add(Activation('softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 150, 150, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 150, 150, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 150, 150, 32)      9248      
_________________________________________________________________
activation_2 (Activation)    (None, 150, 150, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 75, 75, 64)        18496     
_________________________________________________________________
activation_3 (Activation)    (None, 75, 75, 64)       

In [8]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples//32,
    epochs=33,
    shuffle=True,
    validation_data = validation_generator,
    validation_steps = validation_generator.samples//32,
)

Epoch 1/33
79/79 [==============================] - 1604s 20s/step - loss: 2.1696 - accuracy: 0.2370 - val_loss: 1.5813 - val_accuracy: 0.2578
Epoch 2/33
79/79 [==============================] - 1710s 22s/step - loss: 1.5705 - accuracy: 0.3082 - val_loss: 1.3507 - val_accuracy: 0.3871
Epoch 3/33
79/79 [==============================] - 2122s 27s/step - loss: 1.4211 - accuracy: 0.3915 - val_loss: 1.2214 - val_accuracy: 0.4637ETA: 25:58 - loss: 1.5015 - accuracy: 0.35 - ETA: 25:37 - loss:
Epoch 4/33
79/79 [==============================] - 1940s 25s/step - loss: 1.3082 - accuracy: 0.4341 - val_loss: 1.1913 - val_accuracy: 0.5121
Epoch 5/33
79/79 [==============================] - 1968s 25s/step - loss: 1.2009 - accuracy: 0.4898 - val_loss: 1.2020 - val_accuracy: 0.5685
Epoch 6/33
79/79 [==============================] - 1874s 24s/step - loss: 1.1948 - accuracy: 0.5217 - val_loss: 1.1221 - val_accuracy: 0.5403
Epoch 7/33
79/79 [==============================] - 1854s 23s/step - loss: 1.07

In [ ]:
# model.save("3.l98a71.h5")
# from keras.models import load_model
# from keras.models import Sequential
# model = Sequential()
# model = load_model("1.h5")

In [9]:
train_generator.class_indices

{'daisy': 0, 'dandelion': 1, 'rose': 2, 'sunflower': 3, 'tulip': 4}

In [10]:
test_generator = ImageDataGenerator(
    rescale=1./255,
).flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=1,
    class_mode=None,
    shuffle=False,
)

Found 0 images belonging to 0 classes.


In [11]:
test_generator.reset()
pred = model.predict_generator(test_generator, verbose=1)
predicted_class_indices = np.argmax(pred, axis=1)
labels = (train_generator.class_indices)
label = dict((v,k) for k,v in labels.items())
predictions = [label[i] for i in predicted_class_indices]
filenames = [s[5:-4] for s in test_generator.filenames]

AxisError: axis 1 is out of bounds for array of dimension 1

In [ ]:
for idx in range(len(filenames)):
    print('predict  %d' % (int(predictions[idx])))
    print('title    %s' % filenames[idx])
    print('')

In [ ]:
import pandas as pd
from datetime import datetime
now = datetime.today().strftime('%Y%m%d%H%M%S')
mname = type(model).__name__
print(mname, now)
pd.DataFrame({'id': filenames, 'flower_class': predicted_class_indices}).to_csv('JL_'+mname+'.'+now+'model5.csv', index=False)